In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
next = pd.read_excel('EP51_45degree_PR.xlsx',sheet_name='Test Result')
pre = pd.read_excel('EP51_45degree_FFP.xlsx',sheet_name='Test Result')

In [3]:
next.sample(5)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
39,8837,Airtight_table,122,斜率絕對值超出規格_正壓,1/619,0.16
179,17859,Epix_Pro_ATE_TI,4,USB Connection 1,27/792,3.41
15,17800,Epix_Pro_CT,60,GPS_ (L1 + L5) by ref for ep47,13/633,2.05
64,17802,Epix_Pro_FT1,30,GPS (L1+L5),1/615,0.16
106,16510,Epix_Pro_51_M_B,203,Check Vibration Frequency_ FFT(From DAQ),1/589,0.17


In [4]:
pre.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
52,17798,Epix_Pro_AutoATE,28,蜂鳴器V28|Beeper V28P0,7/535,1.31
186,17859,Epix_Pro_ATE_TI,21,D109 (540nm) Brightness and LED Current (20mA),1/529,0.19
72,17801,Epix_Pro_HT,13,頻飄|Drift,4/455,0.88


# Tweak and Filter

Blacklist

In [5]:
ProcessTypeBlackList = 'Pack|Click|RabbitCard|EasyCard|DeleteBundle|ProdScan|FileCopyer'
ItemNameBlackList = 'ESN|Check ProductionMap|Fixture ID'

tweak_filter def

In [6]:
def tweak(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    .query("~ProcessType.str.contains(@ProcessTypeBlackList) and ~ItemName.str.contains(@ItemNameBlackList)")
    )
def tweak_filter(raw):
    return(raw
    .pipe(tweak)
    .query("CountESN > CountESN.quantile(0.05)")
    .query("Retest >= Retest.mean()")
    )

# Merge two phases

In [9]:
pre_next = pd.merge(
    left=tweak_filter(pre), right=tweak(next), how='left',
    left_on=['ItemNameType','ProcessType','Item','ItemName'], right_on=['ItemNameType','ProcessType','Item','ItemName'],
    suffixes=['_pre','_next'])

In [10]:
# pre_next.to_excel('output.xlsx', index=False)

# RetryImprove def

In [10]:
def RetryImprove(df):
    return((100 * (df.Retest_pre.sum() - df.Retest_next.sum()) / df.Retest_pre.sum()).round(3))
RetryImprove(pre_next)

46.504

In [13]:
def RetryDiff(df):
    return(df
     .assign(diff = df.Retest_pre - df.Retest_next)
     [['ProcessType','ItemName','diff']]
     .sort_values('diff', ascending=False)
     .dropna(axis=0)
     .style.bar('diff', color=['red','green'])
     )
RetryDiff(pre_next)

,ProcessType,ItemName,diff
0,Epix_Pro_ATE_TI,AFE Power Consumption data,22.940000
1,Epix_Pro_ATE_TI,Barometer Temperature,11.770000
3,Airtight_table,變形量絕對值超出規格_常壓,11.110000
5,Epix_Pro_ECG_ATE,Motor Resonant Frequency,9.310000
4,Epix_Pro_FT2,ANT_power 2450MHz BY REF (sapphire),6.440000
7,Airtight_table,連續斜率異常_負壓,4.620000
17,Epix_Pro_ALS,Temperature Sensor Temperature,2.600000
20,Epix_Pro_FT1,Analog Front End BIST - Revision 1,2.540000
11,Epix_Pro_HT,電池電壓|Battery Voltage,2.410000
26,Epix_pro_RTESN,Baro Temperature,2.100000


# Total improvement rate

In [11]:
100*((tweak(pre).Retest.sum() - tweak(next).Retest.sum())/ tweak(pre).Retest.sum())

-42.561301477522804